In [26]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, UpSampling2D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping

In [44]:
def get_random_data(data1, data2, low, high, max_samples=100):
    N, H1, W1, C1 = data1.shape
    #_, N1 = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples,))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx]
    return suff_data1, suff_data2

def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1]))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i]
    return new_x[:, :, :, np.newaxis]

def return_datasets(filename):
    data = utils.loadmat('../data/{}'.format(filename))
    trainSet, testSet, valSet = 1, 2, 3
    
    x_train = reformat(data['x'][:, :, data['set']==trainSet ])
    y_train = (data['y'][data['set']==trainSet])
    x_val = reformat(data['x'][:, :, data['set']==valSet])
    y_val = (data['y'][data['set']==valSet])
    x_test = reformat(data['x'][:, :, data['set']==testSet])
    y_test = (data['y'][data['set']==testSet])
    
    x_train, y_train = get_random_data(x_train, y_train, 0, x_train.shape[0], x_train.shape[0])
    x_val, y_val = get_random_data(x_val, y_val, 0, x_val.shape[0], x_val.shape[0])
    x_test, y_test = get_random_data(x_test, y_test, 0, x_test.shape[0], x_test.shape[0])
    
    #print (x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)
    

    #return (X, Y, x_test, y_test)
    return (x_train, y_train, x_val, y_val, x_test, y_test)

In [45]:
def extract_model(activation_func = 'relu'):
    model = Sequential()
    kernel_size=(5)
    stride_size = (2)

    model.add(Conv2D(32, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same', input_shape=(28, 28, 1)))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())

    model.add(UpSampling2D((2, 2)))
#     model.add(Conv2D(64, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
#     model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
   
    
    model.add(Conv2D(96, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    #model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())


    model.add(Flatten())
    model.add(Dense(256, activation=activation_func)) 
    model.add(Dense(10, activation='softmax'))
    
    return model

In [60]:
MAX_EPOCHS = 30
MAX_BATCH_SIZE = 10

In [30]:
#model = extract_model('relu')

In [61]:
x_train, y_train, x_val, y_val, x_test, y_test = return_datasets('digits-jitter.mat')
model = extract_model('relu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, to_categorical(y_val, num_classes=10)), metrics=["accuracy"])
model.fit(x_train, to_categorical(y_train, num_classes=10), epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
accuracy = model.evaluate(x_test, to_categorical(y_test, num_classes=10), verbose=0)[1]
print (accuracy)

Train on 1000 samples
Epoch 1/30
1000/1000 [==============================] - 167s 167ms/sample - loss: 2.1406 - accuracy: 0.2290
Epoch 2/30
1000/1000 [==============================] - 54s 54ms/sample - loss: 1.5204 - accuracy: 0.4660
Epoch 3/30
1000/1000 [==============================] - 1s 910us/sample - loss: 1.1815 - accuracy: 0.5910
Epoch 4/30
1000/1000 [==============================] - 1s 921us/sample - loss: 0.8559 - accuracy: 0.7170
Epoch 5/30
1000/1000 [==============================] - 1s 926us/sample - loss: 0.6607 - accuracy: 0.7780
Epoch 6/30
1000/1000 [==============================] - 1s 934us/sample - loss: 0.5179 - accuracy: 0.8240
Epoch 7/30
1000/1000 [==============================] - 1s 936us/sample - loss: 0.3941 - accuracy: 0.8690
Epoch 8/30
1000/1000 [==============================] - 1s 939us/sample - loss: 0.2968 - accuracy: 0.8940
Epoch 9/30
1000/1000 [==============================] - 1s 989us/sample - loss: 0.3627 - accuracy: 0.8740
Epoch 10/30
1000/1000 

In [54]:
x_test.shape, to_categorical(y_test, num_classes=10).shape, y_test.shape

((500, 28, 28, 1), (500, 10), (500,))

In [53]:
accuracy = model.evaluate(x_val, to_categorical(y_val, num_classes=10), verbose=0)[1]

TypeError: An op outside of the function building code is being passed
a "Graph" tensor. It is possible to have Graph tensors
leak out of the function building context by including a
tf.init_scope in your function building code.
For example, the following function will fail:
  @tf.function
  def has_init_scope():
    my_constant = tf.constant(1.)
    with tf.init_scope():
      added = my_constant * 2
The graph tensor has name: input_iterator:0

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = return_datasets('digits-normal.mat')
model = extract_model('selu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
model.fit(x_train, y_train, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print (accuracy)

In [ ]:
MAX_EPOCHS = 20
MAX_BATCH_SIZE = 10

files = ['digits-jitter.mat', 'digits-normal.mat', 'digits-scaled.mat'] 
for file in files:
    x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)

    model = extract_model('relu')
    model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
    model.fit(x_train, y_train, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
    accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
    print ("Accuracy = {}".format(accuracy))

In [ ]:
MAX_EPOCHS = 20
MAX_BATCH_SIZE = 10

files = ['digits-jitter.mat', 'digits-normal.mat', 'digits-scaled.mat'] 
for file in files:
    x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)

    model = extract_model('selu')
    model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(X[test], to_categorical(Y[test], num_classes=10)), metrics=["accuracy"])
    model.fit(X[train], to_categorical(Y[train], num_classes=10) , epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
    accuracy = model.evaluate(X[test], to_categorical(Y[test], num_classes=10), verbose=0)[1]
    print ("Accuracy = {}".format(accuracy))

In [ ]:
MAX_EPOCHS = 20
MAX_BATCH_SIZE = 10

x_train, y_train, x_val, y_val, x_test, y_test = return_datasets('digits-normal.mat')

model = extract_model('selu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(X[test], to_categorical(Y[test], num_classes=10)), metrics=["accuracy"])
model.fit(X[train], to_categorical(Y[train], num_classes=10) , epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
accuracy = model.evaluate(X[test], to_categorical(Y[test], num_classes=10), verbose=0)[1]
print ("Accuracy = {}".format(accuracy))

In [ ]:
MAX_EPOCHS = 12
MAX_BATCH_SIZE = 256
MAX_TOLERANCE = 5
#x_train, y_train, x_val, y_val, x_test, y_test = return_datasets('digits-jitter.mat')
weight_decay = 1e-4
activation_func = 'selu'

files = ['digits-jitter.mat', 'digits-normal.mat', 'digits-scaled.mat'] 
for file in files:
    X, Y, x_test, y_test = return_datasets(file)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    best_model, best_accuracy = -1, -1
    
    for train, test in kfold.split(X, Y):
        model = extract_model('selu')
        model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(X[test], to_categorical(Y[test], num_classes=10)), metrics=["accuracy"])
        model.fit(X[train], to_categorical(Y[train], num_classes=10) , epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
        accuracy = model.evaluate(X[test], to_categorical(Y[test], num_classes=10), verbose=0)[1]

        if accuracy > best_accuracy:
            print ("Best Accuracy Recorded = {}".format(accuracy))
            best_accuracy = accuracy
            best_model = model

        if best_accuracy > .99:
            break

    print ("Test Set Accuracy for {} = {}".format(file, best_model.evaluate(x_test, to_categorical(y_test, num_classes=10), verbose=0)))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
model.fit(x_train, y_train, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE,  verbose=1)
accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print (accuracy)

In [ ]:

accuracy